In [1]:
import pandas as pd
import numpy as np
import snoop
from pprint import pprint as print
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use("seaborn-whitegrid")
pd.set_option("display.width", 5000)
pd.set_option("display.max_columns", 60)
plt.rcParams["figure.figsize"] = (15, 10)

%load_ext snoop
%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_format = 'svg' 
# show all output not only the last result
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('/Users/glebsokolov/HeadRepo/DataSets/RT_Test.xlsx',
                   skiprows=11)


Построить сводную табличку по площадкам за весь период по метрикам
- Impressions Net
- Clicks Net
- CTR % ( считается как Clicks/Impressions)
- Impressions GIVT Rate % (считается как (Impressions Gross - Impressions Net)/Impression Gross)
- Impressions SIVT Rate% (Impressions SIVT/Impressions Net)
- Clicks GIVT Rate % (считается как (Clicks Gross - Clicks Net)/Clicks Gross)
- Clicks SIVT Rate % (Clicks SIVT/Clicks Net)


In [5]:
df.fillna(0, inplace=True)

cols = df.columns
for num, col in enumerate(df.columns):
    if 'IAB' in col: print(num)
    if 'Recordable' in col: print(num)


10
11


In [6]:
df['Month'] = pd.to_datetime(df['Month'])

grouped = df.groupby('Site Name').sum()
imp_net = grouped['Impressions (Net)']
clicks_net = grouped['Clicks (Net)']
ctr = clicks_net / imp_net
imp_gross = grouped['Impressions (Gross)']
imp_sivt = grouped['Impressions (SIVT)']
imp_givt = (imp_gross - imp_net) / imp_gross
imp_sivt = imp_sivt / imp_net
clicks_givt = (grouped['Clicks (Gross)'] - clicks_net) / clicks_net
clicks_sivt = (grouped['Clicks (SIVT)']) / clicks_net


In [116]:
df

,Month,Campaign Name,Site Name,Placement Name,Placement Type,Placement Dimension,Impressions (Net),Clicks (Net),Impressions (Gross),Clicks (Gross),Total Viewable Impressions (IAB),Total Recordable Impressions,Impressions (SIVT),Clicks (SIVT)
0,2020-10-01,RU_Test_Campaign,Mail.ru,RU_Test_Mail_Display_banner_300x300,In Banner,300x300,4.000,0.000,4.000,0.000,3.000,3.000,0.000,0.000
1,2020-10-01,RU_Test_Campaign,MGID,Test_MGID_Eda,In Banner,300x250,178864.000,107.000,178942.000,114.000,56416.000,176279.000,490.000,0.000
2,2020-10-01,RU_Test_Campaign,OMI RU,OMI_RU_Sync_SAS_new,Tracking,0.000,4522686.000,0.000,4550566.000,0.000,0.000,3396.000,0.000,0.000
3,2020-10-01,RU_Test_Campaign,Tiburon RU,Tiburon_RU_Sync_SAS,Tracking,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000
4,2020-10-01,RU_Test_Campaign_2020,Mail.ru,JLR_test_token_trackers,Tracking,0.000,24.000,31.000,24.000,31.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,2020-12-01,RU_Test_Campaign_2020,Online Interviewer RU,OnIn_RU_Sync_SAS_2020,Tracking,0.000,932019.000,0.000,933802.000,0.000,0.000,0.000,0.000,0.000
163,2020-12-01,RU_Test_Campaign_2020,GFK RU,GFK_RU_Sync_SAS_2020,Tracking,0.000,159841.000,0.000,160170.000,0.000,0.000,0.000,0.000,0.000
164,2020-12-01,Unilever_Fiducia_Test,The Trade Desk_PCP_Unilever,Display_Fiducia_300x250,In Banner,300x250,57.000,9.000,59.000,10.000,0.000,0.000,0.000,0.000
165,2020-12-01,Unilever_Fiducia_Test,The Trade Desk_PCP_Unilever,Display_Fiducia_Video,In-Stream Video,0.000,23.000,0.000,23.000,0.000,0.000,0.000,0.000,0.000


In [13]:
ftab = pd.DataFrame([
    imp_net, clicks_net, ctr, imp_gross, imp_sivt, imp_givt, clicks_givt,
    clicks_sivt
],
                    columns=df['Site Name'].values,
                    index=[
                        'imp_net', 'clicks_net', 'ctr', 'imp_gross',
                        'imp_sivt', 'imp_givt', 'clicks_givt', 'clicks_sivt'
                    ]).transpose().drop_duplicates().fillna(0)
ftab

,imp_net,clicks_net,ctr,imp_gross,imp_sivt,imp_givt,clicks_givt,clicks_sivt
Mail.ru,37.00,63.00,1.70,38.00,0.00,0.03,0.00,0.00
MGID,178872.00,107.00,0.00,178950.00,0.00,0.00,0.07,0.00
OMI RU,33554289.00,0.00,0.00,33631830.00,0.00,0.00,0.00,0.00
Tiburon RU,4262619.00,0.00,0.00,4265156.00,0.00,0.00,0.00,0.00
test.ru,217.00,6.00,0.03,218.00,0.00,0.00,0.17,0.00
InStyle.ru,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00
Adriver RU,4.00,4.00,1.00,10.00,0.00,0.60,1.50,0.00
MTS RU,162022.00,491.00,0.00,162268.00,0.00,0.00,0.14,0.00
Dv360 RU,5684.00,3.00,0.00,5710.00,0.00,0.00,2.33,0.00
Yandex_Search_1,0.00,47.00,inf,0.00,0.00,0.00,0.09,0.00


Построить сводную табличку по площадкам за ДЕКАБРЬ по метрикам
- Impressions Net
- Viewability Rate (Viewable Impressions/Recordable Impressions)

In [9]:
pd.set_option("display.precision", 2)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
grouped_month = df.set_index('Month').groupby(
    [pd.Grouper(freq="M"), 'Site Name']).sum().sort_index().reset_index()
g = grouped_month[[
    'Month', 'Site Name', 'Impressions (Net)', cols[10], cols[11]
]]
g = g[g['Month'] == pd.to_datetime('2020-12-31')]
g['Viewability Rate'] = g[cols[10]] / g[cols[11]]
stab = g[['Site Name', 'Impressions (Net)',
          'Viewability Rate']].set_index('Site Name').fillna(0)
stab.transpose()

Site Name,CTC RU,CarsGuru.ru,Film-pro.ru,GFK RU,M24.ru,MTS RU,Mediatoday RU,MobX RU,OMI RU,OTM RU,Online Interviewer RU,PHD RU,Redllama RU,Rocket MX RU,Roden Media Network,Slickjump RU,The Trade Desk_PCP_Unilever,Tiburon RU,Trading Desk RU,VKontakte (RU),Vesti.ru,Wi-Fi.ru,Yandex.ru,test.ru
Impressions (Net),12122.00,7752.00,2877.00,159841.00,6162.00,0.00,28.00,2.00,12968757.00,3.00,932019.00,0.00,0.00,248020.00,4014.00,0.00,82.00,979964.00,1877915.00,0.00,25.00,43.00,4.00,136.00
Viewability Rate,0.93,0.35,0.84,0.00,0.91,0.00,0.20,0.00,0.00,0.00,0.00,0.00,0.00,0.48,0.58,0.00,0.00,0.00,0.23,0.00,0.88,0.79,0.00,0.83


Взять один сайт со слайда 3 где видимость Viewable Rate ниже среднего и проанализировать на предмет как можно повысить видимость по нему
\
!Подсказка - разложить видимость по размерам баннеров Placement Dimension и проанализировать за счет чего можно поднять общий показатель